In [16]:
import json
from collections import defaultdict
from mongo_handler import MongoHandler

def get_percentage_changes_training_corpus(threshold_perc_change = 300):
    mongo_handler_obj = MongoHandler()
    mongo_handler_obj.connect_to_mongo()
    db = mongo_handler_obj.get_database()
    list_of_input_data = []
    data = db["input_data"].find({"is_used": True, "year": {"$ne": 2021}})
    list_of_ids_to_remove = []
    for item in data:
        list_of_input_data.append(item)
            
    curr_min_standard = 10000
    curr_max_standard = -10000
    curr_min_minmax = 10000
    curr_max_minmax = -10000
    curr_min = 10000000
    curr_max = -10000000
    dict_of_percentage_changes = defaultdict(list)
    for input_data in list_of_input_data:
        
        if input_data["percentage_change"] > threshold_perc_change:
            list_of_ids_to_remove.append(str(input_data["_id"]))
        # if input_data["percentage_change_standard"] < curr_min_standard:
        #     curr_min_standard = input_data["percentage_change_standard"]
        # if input_data["percentage_change_standard"] > curr_max_standard:
        #     curr_max_standard = input_data["percentage_change_standard"]
        # if input_data["percentage_change_min_max"] < curr_min_minmax:
        #     curr_min_minmax = input_data["percentage_change_min_max"]
        # if input_data["percentage_change_min_max"] > curr_max_minmax:
        #     curr_max_minmax = input_data["percentage_change_min_max"]
        if input_data["percentage_change"] < curr_min:
            curr_min= input_data["percentage_change"]
        if input_data["percentage_change"] > curr_max:
            curr_max= input_data["percentage_change"]
            
        dict_of_percentage_changes["percentage_change"].append(input_data["percentage_change"])
        # dict_of_percentage_changes["percentage_change_min_max"].append(input_data["percentage_change_min_max"])
        # dict_of_percentage_changes["percentage_change_standard"].append(input_data["percentage_change_standard"])
            
    return dict_of_percentage_changes, list_of_ids_to_remove
    

In [17]:
dict_of_percentage_changes_train, list_of_ids_to_remove = get_percentage_changes_training_corpus()

In [18]:
list_of_ids_to_remove

['62795156199c6e379f8994a8',
 '62795170c02560533c8994a8',
 '627951862e41d174508994a8',
 '6279518dcd1aecfde58994a8',
 '6279518f6759f8f7cb8994a8',
 '627951a321155cfd1b8994a8',
 '627951a624c07acf3f8994a8']

In [14]:
import plotly.figure_factory as ff
import numpy as np

hist_data = [dict_of_percentage_changes_train["percentage_change"]]

group_labels = ['percentage_change']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

In [27]:
import plotly.figure_factory as ff
import numpy as np

hist_data = [dict_of_percentage_changes_val["percentage_change"], dict_of_percentage_changes_val["percentage_change_standard"], dict_of_percentage_changes_val["percentage_change_min_max"]]

group_labels = ['percentage_change', 'percentage_change_standard', 'percentage_change_min_max']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels)
fig.show()

## Pre-training Adapter results

In [6]:
import pandas as pd

val_df = pd.read_csv("data/pre_training_adapter/run-runs_sec_adapter_maxlen-64_batch-64_lr-5e-05_warmup-0_epoch-100_-tag-eval_loss (1).csv")
train_df = pd.read_csv("data/pre_training_adapter/run-runs_sec_adapter_maxlen-64_batch-64_lr-5e-05_warmup-0_epoch-100_-tag-loss.csv")

df_merged = pd.merge(train_df, val_df, how="inner", on="Step", suffixes=(None, "_val"))

In [8]:
import plotly.express as px

fig = px.line(df_merged, x="Step", y=["Value", "Value_val"], title=f"Pre-training Adapter train vs val loss")
fig.show()